# Commands and State Synchronization in NanoVer

This notebook explores the concepts of commands and state synchronization in NanoVer. These concepts are critical for the function of a multi-user application: 
* **_Commands_** enable clients connecting to the server to run functions on the server
* Synchronisation of the system's **_shared state_** relays any changes made to the server to all connected clients

The implementations of these concepts in NanoVer provide the foundations for interactive multi-user applications (e.g. iMD-VR). 


In this tutorial, you'll learn:
* What a **_command_** is
* How to set up custom commands on a NanoVer server
* What the **_shared state_** is
* How to set shared variables and objects between clients via a NanoVer server

## **Commands**

### What is a command?

In NanoVer, a **_command_** is a function on the server that can be run by a client. These can be used to customize a server,
or create entirely new applications. They are used in lots of places. For example, telling the simulation to pause, play, reset and step uses them! 

Let's create a NanoVer server. All servers and applications that derive from a `NanoverServer` have the ability to run commands.

**NOTE**: One such server is the `OmniRunner`, which we use to perform iMD simulations in NanoVer. For the sake of generality, we will use this simple `NanoverServer` base class to demonstrate how to construct commands, but everything below illustrates how commands work internally within the `OmniRunner`!

In [1]:
from nanover.core import NanoverServer
server = NanoverServer(address='localhost', port=0)

### A Basic Command

Let's set up a very simple command by defining the function that the command will execute: it just prints 'Hello World!'

In [2]:
def hello_world():
    print('Hello World!')

To register it, we have to provide a unique name for the command, which will be used by clients. In this case. we choose the name of the command to be 'hello'

In [3]:
server.register_command('hello', hello_world)

Now let's connect a client to the server.

In [4]:
from nanover.core import NanoverClient

In [5]:
client = NanoverClient.insecure_channel(address=server.address, port=server.port)

We can ask what commands are available on the server.

In [6]:
client.update_available_commands()

{'hello': <nanover.command.command_info.CommandInfo at 0x1048f23c0>}

This tells us that our 'hello' command is registered on the server and can be run by clients. Let's run it! 

In [7]:
client.run_command('hello')

{}

You'll notice that the output of the command was printed in the cell where the server was defined (see above), and it returned an empty dictionary after the cell where we ran the command. This leads us nicely onto more advanced commands...

### Taking Arguments and Returning Results

A command is expected to have the following signature structure:

In [8]:
from typing import Dict
def command(**kwargs) -> Dict[str, object]:
    pass

So this means it can define keyword arguments, and can return a dictionary of objects with string keys. For practical reasons of transmitting over the wire, the dictionary must only contain simple objects, such as numbers, booleans, strings and lists and dictionaries of these things. This is because it is internally converted to a [protobuf Struct](https://protobuf.dev/reference/java/api-docs/com/google/protobuf/Struct.html#:~:text=%60Struct%60%20represents%20a%20structured%20data,is%20represented%20as%20an%20object.), which is similar to a JSON file.

Let's make a more complicated example:

In [9]:
import math

In [10]:
def pythagoras(a=1, b=1):
    c = math.sqrt(a**2 + b**2)
    return {'c': c}

In [11]:
server.register_command('pythagoras', pythagoras)

Let's run this new command - we could update the commands on the client (as per the basic example), but we already know it's been registered

In [12]:
client.run_command('pythagoras')

{'c': 1.4142135623730951}

This time, we receive a dictionary of results, as defined in the method. Let's run the command again, this time passing it some arguments:

In [13]:
client.run_command('pythagoras', a=3, b=4)

{'c': 5.0}

Verifying this result is left as an exercise. 

What happens if we send incorrect arguments? Let's try running the command again, this time passing the argument `c`

In [14]:
import grpc
try:
    client.run_command('pythagoras', c=2)
except grpc.RpcError as e:
    print(e.details())

Exception calling application: pythagoras() got an unexpected keyword argument 'c'


Cool! Even if the client was in a different language, on a different computer, we can see why we messed up!

In defining the `pythagoras` function, we defined some default arguments. It is also possible to define default arguments when registering the command. Let's remove our previous definition, and add it with some defaults. This can be useful if you want to set default behaviour for some method that isn't directly under your control.

In [15]:
server.unregister_command('pythagoras')
server.register_command('pythagoras', pythagoras, {'a':2,'b':2})

In [16]:
client.run_command('pythagoras')

{'c': 2.8284271247461903}

Defining default arguments this way also gives the client some information about what the command accepts:

In [17]:
client.update_available_commands()
client.available_commands['pythagoras'].arguments

{'a': 2.0, 'b': 2.0}

In [18]:
client.close()
server.close()

### Using commands to drive an application

Commands become powerful when they are used to *change* something on the server, making the server interactive. 

The [trajectory viewer](../mdanalysis/mdanalysis_trajectory.ipynb) example demonstrates this. 

## State Synchronisation: the **Shared State** dictionary

NanoVer uses a client-server model, which means clients do not talk to eachother directly. 

Howerver, it is often important to synchronise information across multiple clients, for example the position of the simulation in the VR space, or the current state of one's avatar (e.g. if you're in a menu).

We achieve this in NanoVer via a **_shared state_** dictionary (often referred to simply as the _shared state_) that clients can update. Whenever a client makes a change to this dictionary, the changes are sent to any clients that are listening. 

We're going to demonstrate how the shared state works by creating a server, connecting multiple clients to it and seeing what happens when the shared state is altered by the clients.

To start, let's define a server and connect a client to it.

In [19]:
server = NanoverServer(address='localhost', port=0)

In [20]:
client = NanoverClient.insecure_channel(address='localhost', port=server.port)

The following command makes the client listen to any changes on the shared state dictionary.

In [21]:
client.subscribe_all_state_updates()

We can look at the shared state by calling `copy_state()`. At first, this dictionary is empty:

In [22]:
client.copy_state()

{}

Let's connect a second client to the server.

In [23]:
second_client = NanoverClient.insecure_channel(address='localhost', port=server.port)

In [24]:
second_client.subscribe_all_state_updates()

Now, the first client will make a change to the dictionary, and we'll see that both clients will get the update.

In [25]:
from nanover.state.state_dictionary import DictionaryChange

We can add or update keys, and we can remove a list of keys. In what follows, we set a key `a` to the value 2, and do not remove any keys (since there aren't any)

In [26]:
updates = {'a': 2}
key_removals = []

In [27]:
changes = DictionaryChange(updates=updates,removals=key_removals)

In [28]:
client.attempt_update_state(changes)

True

We have successfully updated the shared state from the first client. Now we'll call `copy_state()` from both clients, and see what happens:

In [29]:
client.copy_state()

{'a': 2.0}

In [30]:
second_client.copy_state()

{'a': 2.0}

We can see that the shared state has been updated with the new value, according to both clients.

The DictionaryChange object took a second argument - keys to remove. We can use that to remove things from the shared state.

In [31]:
changes = DictionaryChange({}, ['a'])
second_client.attempt_update_state(changes)

True

Now, the dictionary is empty. We can check this from both clients:

In [32]:
client.copy_state()

{}

In [33]:
second_client.copy_state()

{}

What if both clients try to update at the same time, or what if I don't want someone else to mess with something?
We handle this by _locking the key_ while changes are made, so only one client can edit fields at a time:

In [34]:
# We aquire a lock on the key 'a' for 10 seconds.
got_lock = client.attempt_update_locks({'a': 10})
print(f'Client has a lock: {got_lock}')
# We attempt to update the locked key from the second client.
successfully_updated = second_client.attempt_update_state(changes)
print(f'Second client updated state: {successfully_updated}')

Client has a lock: True
Second client updated state: False


Here, the second client was unable to update the shared state because the first client had locked the shared state.

The dictionary accepts anything that can be represented as a protobuf Struct, so you can set up complicated things:

In [35]:
changes = DictionaryChange({
    'party':{'pokemon':['charmander','squirtle','bulbasaur','pikachu']},
    'battle':{
        'name':'charmander',
        'type':'fire',
        'level': 7,
        'is_my_favourite': True,
        'abilities':['scratch', 'growl']
    }
}, [])
second_client.attempt_update_state(changes)

True

In [36]:
import pprint # pretty print!
pprint.pprint(client.copy_state())

{'battle': {'abilities': ['scratch', 'growl'],
            'is_my_favourite': True,
            'level': 7.0,
            'name': 'charmander',
            'type': 'fire'},
 'party': {'pokemon': ['charmander', 'squirtle', 'bulbasaur', 'pikachu']}}


## Tidying Up

As ever, once we're finished we should shut down the clients and the server by calling `close()` on them:

In [37]:
client.close()
second_client.close()
server.close()

# Next 



In this notebook, we have learned about **commands** and the **shared state**, two fundamental concepts in NanoVer. Here are some good next steps:
* See practical examples of commands in the [trajectory viewer](../mdanalysis/mdanalysis_trajectory.ipynb)
* Learn more about how [servers](servers.ipynb) are constructed. 
* Look at the [C# client code](https://github.com/IRL2/NarupaUnityPlugin/blob/main/Grpc/GrpcClient.cs) to learn how to run commands from a VR application.